In [1]:
%autosave 30
%load_ext autoreload
%autoreload 2

Autosaving every 30 seconds


In [2]:
cd ..

/home/huawei123/kwx1991442/code-classification


In [3]:
from datasets import load_dataset
from tqdm import tqdm

path_to_data = "data/the-stack-rust-clean/"

In [4]:
dataset = load_dataset("./data/the-stack-rust-clean/data/")

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

In [5]:
iter_dataset = iter(dataset['train'])

def batch_iterator(batch_size=10):
    for _ in tqdm(range(0, len(dataset['train']), batch_size)):
        yield [next(iter_dataset)["content"] for _ in range(batch_size)]

In [6]:
from tokenizers import Tokenizer, decoders, models, normalizers, pre_tokenizers, trainers

tokenizer = Tokenizer(models.BPE(unk_token="[UNK]")) #Unigram())
tokenizer.normalizer = normalizers.Sequence([normalizers.NFKC(), normalizers.Lowercase()])
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
tokenizer.decoder = decoders.ByteLevel()
trainer = trainers.BpeTrainer(
    vocab_size=20_000,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet(),
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
) 

In [7]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

 75%|███████▍  | 66690/89380 [11:52<05:07, 73.75it/s] 

In [ ]:
import os
file = os.path.join(path_to_data, 'tokenizer.json')
tokenizer.save(file)

In [21]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file=file)

In [22]:
fast_tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=20000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=True)